<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0316huggingface_tutorial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers のインストール
! pip install transformers datasets

# 前回のリリースではなく，ソースからインストールする場合は，上記のコマンドをコメントアウトして，下のコマンドをアンコメントしてください。
# To install from source instead of the last release, comment the command above and uncomment the following one.
#! pip install git+https://github.com/huggingface/transformers.git

# 訓練済みモデルを微調整
<!-- # Fine-tune a pretrained model -->

事前学習させたモデルを使用することには，大きなメリットがあります。
計算コストや二酸化炭素排出量を削減し，ゼロから学習させることなく，最先端のモデルを使用することができます。
Transformers は様々な課題のための何千もの事前学習済みモデルへのアクセスを提供します。
事前学習済みモデルを使用する場合，あなたの課題に特化したデータセットで学習させます。
これは微調整 fine-tuning と呼ばれ，非常に強力な学習技法です。
このチュートリアルでは，あなたが選んだ深層学習フレームワークを使って，事前訓練されたモデルを微調整します。
<!-- There are significant benefits to using a pretrained model. 
It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. 
🤗 Transformers provides access to thousands of pretrained models for a wide range of tasks. 
When you use a pretrained model, you train it on a dataset specific to your task. 
This is known as fine-tuning, an incredibly powerful training technique. 
In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice: -->


* 事前訓練モデルを🤗 Transformers [トレーナー Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) で微調整
* TensorFlow で学習したモデルを Keras で微調整
* 事前学習したモデルをネイティブ PyTorch で微調整

<!-- 
* Fine-tune a pretrained model with 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer).
* Fine-tune a pretrained model in TensorFlow with Keras.
* Fine-tune a pretrained model in native PyTorch. -->

<a id='data-processing'></a>

## データセットの準備
<!-- ## Prepare a dataset -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/_BZearw7f0w?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

事前訓練されたモデルを微調整する前に，データセットをダウンロードし，訓練のための準備をします。
前回のチュートリアルでは，学習用のデータを処理する方法を説明しましたが，今回はそのスキルを試す機会です。
<!-- Before you can fine-tune a pretrained model, download a dataset and prepare it for training. 
The previous tutorial showed you how to process data for training, and now you get an opportunity to put those skills to the test!-->

まず [Yelp レビュー](https://huggingface.co/datasets/yelp_review_full) データセットをロードします。
<!-- Begin by loading the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset: -->

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
#dataset[100]
#上ではエラーが出るようです
dataset['train'][100]

Reusing dataset yelp_review_full (/root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

ご存知のように，テキストを処理するためにトークン化器が必要でス。
可変長配列を処理するためのパディング (埋め草) とトランケーション (切断) の先述を含んでいます。
データセットを一度に処理するには 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) メソッドを使用して，データセット全体に前処理関数を適用します。
<!-- As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. 
To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset: -->

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

必要に応じて，全データセットの小さな下位集合を作成して微調整を行い，かかる時間を短縮することも可能です。
<!-- If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes: -->

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<a id='trainer'></a>

## `Trainer` を用いた微調整
<!-- ## Fine-tune with `Trainer` -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Transformers は 🤗 Transformers モデルの訓練に最適化された [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) クラスを提供し，独自の訓練ループを手動で書くことなく簡単に訓練を開始することができます。
Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) API は ロギング，勾配累積，混合精度などの幅広い訓練オプションと機能をサポートしています。
<!-- 🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) class optimized for training 🤗 Transformers models, making it easier to start training without manually writing your own training loop. 
The [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision-->

まず，モデルをロードし，予想されるラベルの数を指定します。
Yelp Review の[dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields) から，ラベル は 5 つあることがわかります。.
<!-- Start by loading your model and specify the number of expected labels. 
From the Yelp Review [dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields), you know there are five labels: -->

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

<Tip>

事前学習された重みの一部が使用されておらず，一部の重みがランダムに初期化されているという警告が表示されます。
これは全く正常な状態ですので，安心してください。
BERT モデルの事前学習されたヘッドは破棄され，ランダムに初期化された分類ヘッドに置き換えられます。
あなたは系列分類課題でこの新しいモデルヘッドを微調整し，事前学習されたモデルの知識をそれに転送します。

<!-- You will see a warning about some of the pretrained weights not being used and some weights being randomly initialized. 
Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. 
You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it. -->

</Tip>

### ハイパーパラメータの訓練
<!-- ### Training hyperparameters -->

次に [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) クラスを作成します。
このクラスには，調整可能なすべてのハイパーパラメータと，異なる訓練オプションを有効にするためのフラグが含まれています。
このチュートリアルでは，デフォルトの訓練 [ハイパーパラメータ hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) で始めることが可能ですが，これらを自由に試して最適設定を見つけることができます。
<!-- Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. 
For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.-->

訓練のチェックポイントを保存する場所を指定します。
<!-- Specify where to save the checkpoints from your training: -->

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### 計量
<!-- ### Metrics -->

[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) は学習中のモデルの性能を自動的に評価するものではありません。
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) に計量を計算しレポートする関数を渡す必要があります。
🤗 データセットライブラリは簡潔な [`accuracy`](https://huggingface.co/metrics/accuracy) 関数を提供しており， `load_metric` (詳細はこちらの [チュートリアル](https://huggingface.co/docs/datasets/metrics.html) 参照) 関数でロードすることができます。
<!-- [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. 
You will need to pass [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. 
The 🤗 Datasets library provides a simple [`accuracy`](https://huggingface.co/metrics/accuracy) function you can load with the `load_metric` (see this [tutorial](https://huggingface.co/docs/datasets/metrics.html) for more information) function: -->

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trainer.train()

<a id='keras'></a>

## Fine-tune with Keras

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/rnTGBy2ax1c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

🤗 Transformers models also supports training in TensorFlow with the Keras API. You only need to make a few changes before you can fine-tune.

### Convert dataset to TensorFlow format

The [DefaultDataCollator](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DefaultDataCollator) assembles tensors into a batch for the model to train on. Make sure you specify `return_tensors` to return TensorFlow tensors:

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

<Tip>

[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) uses [DataCollatorWithPadding](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DataCollatorWithPadding) by default so you don't need to explicitly specify a data collator.

</Tip>

Next, convert the tokenized datasets to TensorFlow datasets with the [`to_tf_dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.to_tf_dataset) method. Specify your inputs in `columns`, and your label in `label_cols`:

In [ ]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

### Compile and fit

Load a TensorFlow model with the expected number of labels:

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Then compile and fine-tune your model with [`fit`](https://keras.io/api/models/model_training_apis/) as you would with any other Keras model:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

<a id='pytorch_native'></a>

## Fine-tune in native PyTorch

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Dh9CL8fyG80?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) takes care of the training loop and allows you to fine-tune a model in a single line of code. For users who prefer to write their own training loop, you can also fine-tune a 🤗 Transformers model in native PyTorch.

At this point, you may need to restart your notebook or execute the following code to free some memory:

In [ ]:
del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

Next, manually postprocess `tokenized_dataset` to prepare it for training.

1. Remove the `text` column because the model does not accept raw text as an input:

    ```py
    >>> tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    ```

2. Rename the `label` column to `labels` because the model expects the argument to be named `labels`:

    ```py
    >>> tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    ```

3. Set the format of the dataset to return PyTorch tensors instead of lists:

    ```py
    >>> tokenized_datasets.set_format("torch")
    ```

Then create a smaller subset of the dataset as previously shown to speed up the fine-tuning:

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### DataLoader

Create a `DataLoader` for your training and test datasets so you can iterate over batches of data:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Load your model with the number of expected labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

### Optimizer and learning rate scheduler

Create an optimizer and learning rate scheduler to fine-tune the model. Let's use the [`AdamW`](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) optimizer from PyTorch:

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Create the default learning rate scheduler from [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer):

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

Lastly, specify `device` to use a GPU if you have access to one. Otherwise, training on a CPU may take several hours instead of a couple of minutes.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

<Tip>

Get free access to a cloud GPU if you don't have one with a hosted notebook like [Colaboratory](https://colab.research.google.com/) or [SageMaker StudioLab](https://studiolab.sagemaker.aws/).

</Tip>

Great, now you are ready to train! 🥳

### Training loop

To keep track of your training progress, use the [tqdm](https://tqdm.github.io/) library to add a progress bar over the number of training steps:

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### Metrics

Just like how you need to add an evaluation function to [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer), you need to do the same when you write your own training loop. But instead of calculating and reporting the metric at the end of each epoch, this time you will accumulate all the batches with [`add_batch`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=add_batch#datasets.Metric.add_batch) and calculate the metric at the very end.

In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<a id='additional-resources'></a>

## Additional resources

For more fine-tuning examples, refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) includes scripts
  to train common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) contains various notebooks on how to fine-tune a model for specific tasks in PyTorch and TensorFlow.